In [3]:
# =======================
# Hardware / software environment (CPU/GPU/RAM/torch/cuda/cudnn)
# =======================
import os
import platform
import subprocess
import re
from datetime import datetime

def _run_cmd(cmd):
    """Run command and return stdout (str). Never raises; returns '' on failure."""
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
        return out.strip()
    except Exception:
        return ""

def get_cpu_model_linux():
    # Try /proc/cpuinfo first (most reliable on Linux)
    txt = _run_cmd(["bash", "-lc", "cat /proc/cpuinfo | grep -m1 'model name'"])
    if txt:
        # model name : Intel(R) Xeon...
        return txt.split(":", 1)[-1].strip()
    # Fallback: lscpu
    txt = _run_cmd(["bash", "-lc", "lscpu | grep -m1 'Model name'"])
    if txt:
        return txt.split(":", 1)[-1].strip()
    return "N/A"

def get_total_ram_gb_linux():
    txt = _run_cmd(["bash", "-lc", "grep -m1 MemTotal /proc/meminfo"])
    if not txt:
        return "N/A"
    # MemTotal:       32708940 kB
    m = re.search(r"MemTotal:\s+(\d+)\s+kB", txt)
    if not m:
        return "N/A"
    kb = int(m.group(1))
    gb = kb / (1024**2)  # kB -> GB (GiB-ish)
    return f"{gb:.1f} GB"

def get_gpu_info_nvidia_smi():
    """
    Returns list of dicts: [{"name":..., "vram_gb":..., "driver":..., "cuda_reported":...}, ...]
    If not available, returns [].
    """
    # Check nvidia-smi availability
    smi = _run_cmd(["bash", "-lc", "command -v nvidia-smi"])
    if not smi:
        return []

    # Query name and memory
    q = "nvidia-smi --query-gpu=name,memory.total,driver_version --format=csv,noheader,nounits"
    out = _run_cmd(["bash", "-lc", q])
    if not out:
        return []

    gpus = []
    for line in out.splitlines():
        parts = [p.strip() for p in line.split(",")]
        if len(parts) >= 3:
            name, mem_mb, driver = parts[0], parts[1], parts[2]
            try:
                vram_gb = float(mem_mb) / 1024.0
                vram_str = f"{vram_gb:.1f} GB"
            except Exception:
                vram_str = "N/A"
            gpus.append({"name": name, "vram": vram_str, "driver": driver})

    # Also try to read "CUDA Version" reported by nvidia-smi header
    header = _run_cmd(["bash", "-lc", "nvidia-smi | head -n 3"])
    cuda_ver = "N/A"
    m = re.search(r"CUDA Version:\s*([\d.]+)", header)
    if m:
        cuda_ver = m.group(1)

    for g in gpus:
        g["cuda_reported_by_driver"] = cuda_ver
    return gpus

def get_torch_cuda_cudnn_info():
    info = {
        "python": platform.python_version(),
        "pytorch": "N/A",
        "cuda_available": "N/A",
        "torch_cuda": "N/A",
        "cudnn": "N/A",
        "cudnn_enabled": "N/A",
        "gpu_count_torch": "N/A",
    }
    try:
        import torch
        info["pytorch"] = torch.__version__
        info["cuda_available"] = bool(torch.cuda.is_available())
        info["torch_cuda"] = torch.version.cuda if torch.version.cuda is not None else "N/A"
        info["cudnn"] = torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else "N/A"
        info["cudnn_enabled"] = bool(torch.backends.cudnn.enabled)
        info["gpu_count_torch"] = torch.cuda.device_count() if torch.cuda.is_available() else 0
    except Exception:
        pass
    return info

def get_os_info():
    # More detailed OS info for Linux
    os_pretty = _run_cmd(["bash", "-lc", "cat /etc/os-release 2>/dev/null | grep -m1 PRETTY_NAME"])
    if os_pretty:
        return os_pretty.split("=", 1)[-1].strip().strip('"')
    return f"{platform.system()} {platform.release()}"

def collect_environment():
    env = {}
    env["timestamp"] = datetime.now().isoformat(timespec="seconds")
    env["os"] = get_os_info()
    env["cpu"] = get_cpu_model_linux()
    env["ram"] = get_total_ram_gb_linux()

    gpus = get_gpu_info_nvidia_smi()
    env["gpus"] = gpus  # list
    env["torch"] = get_torch_cuda_cudnn_info()
    return env

env = collect_environment()

# --- Print in a table-like way ---
print("=== Environment summary ===")
print("Time:", env["timestamp"])
print("OS:", env["os"])
print("CPU:", env["cpu"])
print("RAM:", env["ram"])

if env["gpus"]:
    print("\nGPU(s) (nvidia-smi):")
    for i, g in enumerate(env["gpus"]):
        print(f"  - GPU {i}: {g['name']} ({g['vram']} VRAM), driver {g['driver']}, CUDA(driver) {g['cuda_reported_by_driver']}")
else:
    print("\nGPU(s): N/A (no nvidia-smi or no NVIDIA GPU detected)")



=== Environment summary ===
Time: 2026-02-09T23:42:39
OS: Ubuntu 22.04.5 LTS
CPU: Intel(R) Xeon(R) Gold 5215 CPU @ 2.50GHz
RAM: 125.4 GB

GPU(s) (nvidia-smi):
  - GPU 0: Tesla T4 (15.0 GB VRAM), driver 535.247.01, CUDA(driver) 12.2
  - GPU 1: Tesla T4 (15.0 GB VRAM), driver 535.247.01, CUDA(driver) 12.2
  - GPU 2: Tesla T4 (15.0 GB VRAM), driver 535.247.01, CUDA(driver) 12.2
  - GPU 3: Tesla T4 (15.0 GB VRAM), driver 535.247.01, CUDA(driver) 12.2
